# Unit 5 - Financial Planning

In [5]:
!pip install alpaca-trade-api
!pip install python-dotenv

In [6]:
import os
import requests
import pandas as pd
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
from datetime import datetime, timedelta

%matplotlib inline

In [7]:
# Keys for interacting with the Alpaca Trade API are contained within the
# the project's environment
load_dotenv()

False

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [8]:
monthly_income = 12000.00

#::::: CRYPTO ASSETS (Bitcoin and Ethereum) :::::
my_btc = 1.2
my_eth = 5.3

In [9]:
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

In [10]:
# `alternative.me`'s API requires indexing based off arbitrary values, hence the
# `["1"]` index
btc_current_price = float(requests.get(btc_url).json()["data"]["1"]["quotes"]["USD"]["price"])
btc_current_price

34540.0

In [11]:
# `alternative.me`'s API requires indexing based off arbitrary values, hence the
# `["1027"]` index
eth_current_price = float(requests.get(eth_url).json()["data"]["1027"]["quotes"]["USD"]["price"])
eth_current_price

1821.8

In [12]:
my_btc_value = btc_current_price * my_btc
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")

my_eth_value = eth_current_price * my_eth
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

The current value of your 1.2 BTC is $41448.00
The current value of your 5.3 ETH is $9655.54


### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [13]:
#::::: STOCK (iShares Core US Aggregate Bond ETF & SPDR S&P 500 ETF Trust) :::::
my_agg = 200
my_spy = 50

In [14]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
print(alpaca_api_key)

alpaca_secret_key = os.getenv("ALPACA_API_SECRET_KEY")
print(alpaca_secret_key)


alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "v2")

None
None


ValueError: ignored

In [16]:
# Alpaca Trade API requires dates to passed in the ISO format
date = pd.Timestamp("2023-10-23", tz="America/New_York").isoformat()


tickers = ["AGG", "SPY"]


# Alpaca Trade API requires a timeframe parameter in the format as rougly
# shown by the following string
timeframe = "1Day"


# Using the provided `get_bars` function in the `alpaca_trade_package` to
# request the API
# https://alpaca.markets/deprecated/docs/api-documentation/api-v2/market-data/alpaca-data-api-v1/bars/
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = date,
    end = date
).df

# Seperate the combined given data as located in `df_portfolio` into two
# DataFrame objects to improve formatting of the resulting DataFrame
AGG = df_portfolio[df_portfolio["symbol"]=="AGG"].drop("symbol", axis = 1)
SPY = df_portfolio[df_portfolio["symbol"]=="SPY"].drop("symbol", axis = 1)


# Combine the two seperated DataFrames into one for usability
df_portfolio = pd.concat([AGG, SPY], axis = 1, keys = ["AGG", "SPY"])


df_portfolio

NameError: ignored

In [ ]:
# `df_portfolio` contains a singular row for both AGG & SPY's closing price
# thus we can use indexing to retrieve the value (1 row = index value 0)

agg_close_price = df_portfolio["AGG"]["close"][0]
print(f"Current AGG closing price: ${agg_close_price}")


spy_close_price = df_portfolio["SPY"]["close"][0]
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
my_agg_value = my_agg * agg_close_price
my_spy_value = my_spy * spy_close_price


print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")

### Savings Health Analysis

In [ ]:
# The "total value" of crypto in dollars
crypto_tv = my_btc_value + my_eth_value
print(crypto_tv)

# The "total value" of stock in dollars
stock_tv = my_agg_value + my_spy_value
print(stock_tv)


# Marshalling the `amounts` of Crypto and Stock ordered for a DataFrame
savings_data = [crypto_tv, stock_tv]
# The provided `index` parameter names the two columns, crypto and stock/bond,
# respectively
savings_df = pd.DataFrame(savings_data,
                          columns = ["amount"],
                          index = ["crypto", "stock/bond"])


display(savings_df)

In [ ]:
savings_pie = savings_df.plot(kind = "pie",
                              title = "Asset Allocation",
                              shadow = True,
                              subplots = True)

In [ ]:
# Generally prescribed financial advice suggest that a emergency fund should
# be three months of one's income
emergency_fund = monthly_income * 3


total_savings = crypto_tv + stock_tv
# Simple test cases to evaluate `if` statement correctness
# total_savings = monthly_income * 3
# total_savings = 420
print(total_savings)


# Convey to a user their status in saving for an Emergency Budget
if total_savings > emergency_fund:
  print("Congratulations! Your total savings is greater than your emergency fund goal 🎉")
elif total_savings == emergency_fund:
  print("Congratulations! You've just reached your emergency fund goal ✅")
else:
  print(f"Amount until goal is reached: ${(emergency_fund - total_savings):0.2f}")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [17]:
# Five years worth of data to create an effective Monte Carlo simulation
start_date = pd.Timestamp('2018-10-23', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-10-23', tz='America/New_York').isoformat()


five_year_stock_portfolio_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


# Seperate the combined given data as located in `df_portfolio` into two
# DataFrame objects to improve formatting of the resulting DataFrame
AGG = five_year_stock_portfolio_df[five_year_stock_portfolio_df["symbol"] == "AGG"].drop("symbol", axis = 1)
SPY = five_year_stock_portfolio_df[five_year_stock_portfolio_df["symbol"] == "SPY"].drop("symbol", axis = 1)


# Combine the two seperated DataFrames into one for usability
five_year_stock_portfolio_df = pd.concat([AGG, SPY], axis = 1, keys = ["AGG", "SPY"])


five_year_stock_portfolio_df.sample(5)

NameError: ignored

In [ ]:
# Configuring simulation to forecast 30 years of cumulative returns
sim = MCSimulation(five_year_stock_portfolio_df,
                   weights = [.40, .60],
                   num_simulation = 500,
                   num_trading_days = 252 * 30)

In [ ]:
sim.calc_cumulative_return()

In [ ]:
sim.plot_simulation()

In [ ]:
sim.plot_distribution()

### Retirement Analysis

In [ ]:
sim.summarize_cumulative_return()

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
initial_investment = 20000


sim.confidence_interval
#print(sim.confidence_interval)


# Confidence intervalues are provided in a series, thus we have to use the
# key values for the given series to access the calculated CI Upper/Lower
ci_lower = initial_investment * sim.confidence_interval["95% CI Lower"]
ci_upper = initial_investment * sim.confidence_interval["95% CI Upper"]
print(f"There is a 95% chance that an initial investment of \n ${initial_investment} in the portfolio"
      f"over the next 30 years will end \n within in the range of"
      f"${ci_lower:0.2f} and ${ci_upper:0.2f}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
initial_investment = 20000 * 1.5


ci_lower = initial_investment * sim.confidence_interval["95% CI Lower"]
ci_upper = initial_investment * sim.confidence_interval["95% CI Upper"]
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower:0.2f} and ${ci_upper:0.2f}")